<img src="https://github.com/alan-barzilay/NLPortugues/blob/master/imagens/logo_nlportugues.png?raw=true"  style="height:65%" align="right">


# Lista 6 - LSTM&GRU 
**Nome:** André Barbosa

**Numero Usp:** 7971751









______________




O objetivo desta lista é fazer com que vocês treinem um modelo de análise de sentimentos utilizando GRU's e LSTM's. Essa lista é semelhante a lista 03 onde aprendemos a usar embeddings e onde você ja recebeu a arquitetura do seu modelo quase pronta. A diferença é que desta vez você ira construir sozinho sua rede e utilizará as camadas que acabamos de aprender: LSTM e GRU.
 
Essa tambêm será a primeira rede recorrente que montaremos, portanto a tokenização será ligeiramente diferente (por exemplo o padding não é mais necessário.)

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
tf.__version__

'2.2.1'

## Importando os dados como um dataframe

Para esta lista nós utilizaremos um recorte de 10 mil linhas do dataset **B2W-Reviews01** que consiste em avaliações de mais de 130k compras online no site Americanas.com e [esta disponivel no github](https://github.com/b2wdigital/b2w-reviews01) sob a licensa CC BY-NC-SA 4.01.

In [3]:
b2wCorpus = pd.read_csv("data/b2w-10k.csv")
b2wCorpus.head()

,submission_date,reviewer_id,product_id,product_name,product_brand,site_category_lv1,site_category_lv2,review_title,overall_rating,recommend_to_a_friend,review_text,reviewer_birth_year,reviewer_gender,reviewer_state,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,2018-01-01 00:11:28,d0fb1ca69422530334178f5c8624aa7a99da47907c44de...,132532965,Notebook Asus Vivobook Max X541NA-GO472T Intel...,NaN,Informática,Notebook,Bom,4,Yes,Estou contente com a compra entrega rápida o ú...,1958,F,RJ,NaN,NaN,NaN,NaN,NaN
1,2018-01-01 00:13:48,014d6dc5a10aed1ff1e6f349fb2b059a2d3de511c7538a...,22562178,Copo Acrílico Com Canudo 500ml Rocie,NaN,Utilidades Domésticas,"Copos, Taças e Canecas","Preço imbatível, ótima qualidade",4,Yes,"Por apenas R$1994.20,eu consegui comprar esse ...",1996,M,SC,NaN,NaN,NaN,NaN,NaN
2,2018-01-01 00:26:02,44f2c8edd93471926fff601274b8b2b5c4824e386ae4f2...,113022329,Panela de Pressão Elétrica Philips Walita Dail...,philips walita,Eletroportáteis,Panela Elétrica,ATENDE TODAS AS EXPECTATIVA.,4,Yes,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...,1984,M,SP,NaN,NaN,NaN,NaN,NaN
3,2018-01-01 00:35:54,ce741665c1764ab2d77539e18d0e4f66dde6213c9f0863...,113851581,Betoneira Columbus - Roma Brinquedos,roma jensen,Brinquedos,Veículos de Brinquedo,presente mais que desejado,4,Yes,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...,1985,F,SP,NaN,NaN,NaN,NaN,NaN
4,2018-01-01 01:00:28,7d7b6b18dda804a897359276cef0ca252f9932bf4b5c8e...,131788803,"Smart TV LED 43"" LG 43UJ6525 Ultra HD 4K com C...",lg,TV e Home Theater,TV,"Sem duvidas, excelente",5,Yes,"A entrega foi no prazo, as americanas estão de...",1994,M,MG,NaN,NaN,NaN,NaN,NaN


In [4]:
b2wCorpus["review_text"]

0       Estou contente com a compra entrega rápida o ú...
1       Por apenas R$1994.20,eu consegui comprar esse ...
2       SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...
3       MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...
4       A entrega foi no prazo, as americanas estão de...
                              ...                        
9994    Celular muito rápido, com processador e armaze...
9995    achei o produto muito frágil, o material veio ...
9996    Uma porcaria pois ñ recebi ñ recomendo pra nin...
9997    Maquina excelente,super pratica. recomendo.ent...
9998    Agradeço pelo compromisso, obrigado. ,...........
Name: review_text, Length: 9999, dtype: object


## Pré-processamento 
# <font color='blue'>Questão 1 </font>
Copie suas etapas de préprocessamento da lista 03, ou seja, selecione apenas as colunas relevantes ("review_text" e "recommend_to_a_friend"), converta a coluna "review_text" de uma coluna de `str` para uma coluna de `int` e separe os dados em teste e treino.


In [5]:
# Seu código aqui
df = b2wCorpus[["review_text", "recommend_to_a_friend"]].copy()
df['recommend_to_a_friend'] = df['recommend_to_a_friend'].map({'Yes': 1, 'No':0})

df.head()

,review_text,recommend_to_a_friend
0,Estou contente com a compra entrega rápida o ú...,1
1,"Por apenas R$1994.20,eu consegui comprar esse ...",1
2,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...,1
3,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...,1
4,"A entrega foi no prazo, as americanas estão de...",1


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df[['review_text']],
                                                    df[['recommend_to_a_friend']],
                                                    random_state=42,
                                                    test_size=.2)

## Tokenizando




# <font color='blue'>Questão 2 </font>
Utilizando a camada [`TextVectorization`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/TextVectorization) tokenize os inputs.
Declare a camada e então chame a função `adapt()` no seu conjunto de treino para adequar o seu vocabulário aos reviews. 

Note que o uso de padding não é mais necessario.

In [7]:
# Seu código aqui
tokenizer_layer = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=350, standardize='lower_and_strip_punctuation',
    split='whitespace', ngrams=None, output_mode="int",
    output_sequence_length=None, pad_to_max_tokens=False)
tokenizer_layer.adapt(X_train.values)

## LSTM&GRU

Agora vamos juntar a camada do tokenizador a nossa camada [Embedding](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding) e definir o resto de nosso modelo.

#  <font color='blue'>Questão 3 </font>

a) Defina, compile, treine e avalie seu modelo, utilize camadas  [LSTM](https://keras.io/api/layers/recurrent_layers/lstm/).
Atenção a dimensão do input da camada de embedding, lembre se que < OOV > e < PAD > possuem seus próprios tokens.
 
 





**<font color='red'> Sua resposta aqui </font>**

In [8]:
# Seu código aqui
VOCAB_SIZE = len(tokenizer_layer.get_vocabulary())+2 #OOV + PAD
model = tf.keras.Sequential([    
    tokenizer_layer,
    tf.keras.Input(shape=(VOCAB_SIZE,)),
    tf.keras.layers.Embedding(
        VOCAB_SIZE,
        128
    ),
    tf.keras.layers.LSTM(units=128,
                         recurrent_dropout=0.1),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

In [9]:
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
my_callbacks = [tf.keras.callbacks.EarlyStopping(patience=2)]
model.compile(optimizer=opt, loss="binary_crossentropy", metrics=["acc"])

In [10]:
_ = model.fit(X_train.values,y_train.values,
              epochs=10,
              callbacks=my_callbacks,
              validation_data=(
                  X_test.values, y_test.values)
         )

Epoch 1/10
250/250 [==============================] - 36s 143ms/step - loss: 0.5007 - acc: 0.7731 - val_loss: 0.3894 - val_acc: 0.8265
Epoch 2/10
250/250 [==============================] - 37s 146ms/step - loss: 0.4364 - acc: 0.7718 - val_loss: 0.3911 - val_acc: 0.7770
Epoch 3/10
250/250 [==============================] - 37s 149ms/step - loss: 0.4408 - acc: 0.8339 - val_loss: 0.4045 - val_acc: 0.8535


 
b) Como foi a performance desta rede em comparação a da lista 3?

Comparando a acurácia de teste, a obtida coma rede da lista 3 foi de 0.8835, rodando em 4 épocas, sendo que cada época rodou em 4s 16ms. A rede LSTM teve um desempenho _pior_, chegando a 0.8535 de acurácia, e levando quase 10 vezes a mais por época.

#  <font color='blue'>Questão 4 </font>

a) Defina, compile, treine e avalie seu modelo, utilize camadas [GRU](https://keras.io/api/layers/recurrent_layers/gru/).
Atenção a dimensão do input da camada de embedding, lembre se que < OOV > e < PAD > possuem seus próprios tokens.
 
 
 
b) Como foi a performance desta rede em comparação a da lista 3?


**<font color='red'> Sua resposta aqui </font>**

In [11]:
# Seu código aqui# Seu código aqui
VOCAB_SIZE = len(tokenizer_layer.get_vocabulary())+2 #OOV + PAD
model = tf.keras.Sequential([    
    tokenizer_layer,
    tf.keras.Input(shape=(VOCAB_SIZE,)),
    tf.keras.layers.Embedding(
        VOCAB_SIZE,
        128
    ),
    tf.keras.layers.GRU(units=128,
                         recurrent_dropout=0.1),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
my_callbacks = [tf.keras.callbacks.EarlyStopping(patience=2)]
model.compile(optimizer=opt, loss="binary_crossentropy", metrics=["acc"])
_ = model.fit(X_train.values,y_train.values,
              epochs=10,
              callbacks=my_callbacks,
              validation_data=(
                  X_test.values, y_test.values)
         )

Epoch 1/10
250/250 [==============================] - 31s 125ms/step - loss: 0.5694 - acc: 0.7451 - val_loss: 0.5534 - val_acc: 0.7620
Epoch 2/10
250/250 [==============================] - 31s 123ms/step - loss: 0.5673 - acc: 0.7475 - val_loss: 0.5472 - val_acc: 0.7635
Epoch 3/10
250/250 [==============================] - 31s 123ms/step - loss: 0.5621 - acc: 0.7513 - val_loss: 0.5510 - val_acc: 0.7625
Epoch 4/10
250/250 [==============================] - 30s 121ms/step - loss: 0.4170 - acc: 0.8125 - val_loss: 0.2794 - val_acc: 0.8865
Epoch 5/10
250/250 [==============================] - 31s 122ms/step - loss: 0.2848 - acc: 0.8831 - val_loss: 0.2547 - val_acc: 0.8980
Epoch 6/10
250/250 [==============================] - 32s 128ms/step - loss: 0.2581 - acc: 0.8986 - val_loss: 0.2532 - val_acc: 0.8970
Epoch 7/10
250/250 [==============================] - 31s 123ms/step - loss: 0.2461 - acc: 0.9051 - val_loss: 0.2499 - val_acc: 0.9010
Epoch 8/10
250/250 [==============================] - 3

A rede GRU teve um desempenho _melhor_ do que a rede do exercício 3, chegando a 0.9010 e sendo um pouco mais _rápida_ que a LSTM

## Redes Bi-direcionais
#  <font color='blue'>Questão 5 </font>

a) Defina, compile, treine e avalie um novo modelo que utilize contexto em ambas as direções usando a camada [`Bidirectional()`](https://keras.io/api/layers/recurrent_layers/bidirectional/), seja com camadas GRU ou LSTM.


b) Como foi sua performance em relação as questões anteriores com contexto unidirecional?

**<font color='red'> Sua resposta aqui </font>**

In [12]:
# Seu código aqui
VOCAB_SIZE = len(tokenizer_layer.get_vocabulary())+2 #OOV + PAD
GRU_layer =  tf.keras.layers.GRU(units=128, recurrent_dropout=0.1)
model = tf.keras.Sequential([    
    tokenizer_layer,
    tf.keras.Input(shape=(VOCAB_SIZE,)),
    tf.keras.layers.Embedding(
        VOCAB_SIZE,
        128
    ),
    tf.keras.layers.Bidirectional(GRU_layer),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
my_callbacks = [tf.keras.callbacks.EarlyStopping(patience=2)]
model.compile(optimizer=opt, loss="binary_crossentropy", metrics=["acc"])
_ = model.fit(X_train.values,y_train.values,
              epochs=10,
              callbacks=my_callbacks,
              validation_data=(
                  X_test.values, y_test.values)
         )

Epoch 1/10
250/250 [==============================] - 48s 190ms/step - loss: 0.3614 - acc: 0.8404 - val_loss: 0.2773 - val_acc: 0.8750
Epoch 2/10
250/250 [==============================] - 49s 198ms/step - loss: 0.2806 - acc: 0.8856 - val_loss: 0.2633 - val_acc: 0.8830
Epoch 3/10
250/250 [==============================] - 48s 193ms/step - loss: 0.2590 - acc: 0.8959 - val_loss: 0.2616 - val_acc: 0.8865
Epoch 4/10
250/250 [==============================] - 48s 192ms/step - loss: 0.2473 - acc: 0.9014 - val_loss: 0.2661 - val_acc: 0.8875
Epoch 5/10
250/250 [==============================] - 50s 199ms/step - loss: 0.2361 - acc: 0.9071 - val_loss: 0.2547 - val_acc: 0.8950
Epoch 6/10
250/250 [==============================] - 48s 192ms/step - loss: 0.2277 - acc: 0.9126 - val_loss: 0.2601 - val_acc: 0.8965
Epoch 7/10
250/250 [==============================] - 48s 194ms/step - loss: 0.2228 - acc: 0.9132 - val_loss: 0.2503 - val_acc: 0.9010
Epoch 8/10
250/250 [==============================] - 4

A rede GRU bi direcional teve um desempenho levemente _melhor_ do que as demais , chegando a 0.9181 de acurácia de treino. Contudo, ela teve um leve overfitting, uma vez que a acurácia de teste mais alta foi de 0.9030. Além disso, a rede que levou mais tempo para rodar.